In [38]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("American-Airlines") \
    .getOrCreate()

# Path to your CSV file
csv_file_path = "/Users/mika/ML-Big-Data/Project/Dataset/2008.csv"

# Read the CSV file into a DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df.show(5)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2008|    1|         3|        4|   2003|      1955|   2211|      2225|           WN|      335

In [39]:
df.count()

7009728

In [40]:
df = df.drop("Year", "TailNum", "CancellationCode", "TailNum", "DayOfMonth", "FlightNum", "DepTime", "CRSDepTime", "CRSArrTime")

In [41]:
df.show(5)

+-----+---------+-------+-------------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+--------+------------+------------+--------+-------------+-----------------+
|Month|DayOfWeek|ArrTime|UniqueCarrier|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+-----+---------+-------+-------------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+--------+------------+------------+--------+-------------+-----------------+
|    1|        4|   2211|           WN|              128|           150|    116|     -14|       8|   IAD| TPA|     810|     4|      8|        0|       0|          NA|          NA|      NA|           NA|               NA|
|    1|        4|   1002|           WN|              128|           145|    113|       2|      19|   IAD| TPA|     8

In [42]:
cancellation_count = df.filter(df.Cancelled == 1).count()

# Print the result
print(f"Number of rows with cancellation = 1: {cancellation_count}")

Number of rows with cancellation = 1: 137434


In [43]:

# Remove all cancelled flights
df = df.filter(df.Cancelled != 1)


In [44]:
df.count()

6872294

In [45]:
df = df.drop("Cancelled")

In [46]:
from pyspark.sql.functions import col, isnan, when, count

# Count NULL or NaN values for each column
null_counts = df.select(
    [count(when(col(c).isNull() | isnan(col(c)) | (col(c) == "NA"), c)).alias(c) for c in df.columns]
)

# Show the result
null_counts.show()

+-----+---------+-------+-------------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+--------+------------+------------+--------+-------------+-----------------+
|Month|DayOfWeek|ArrTime|UniqueCarrier|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+-----+---------+-------+-------------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+--------+------------+------------+--------+-------------+-----------------+
|    0|        0|  14215|            0|            17265|           598|  17265|   17265|       0|     0|   0|       0| 14215|      0|       0|     5347559|     5347559| 5347559|      5347559|          5347559|
+-----+---------+-------+-------------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+--------+------------+

In [47]:
df = df.filter(
    (df.ArrTime != "NA") & 
    (df.ActualElapsedTime != "NA") & 
    (df.CRSElapsedTime != "NA") & 
    (df.ArrDelay != "NA") & 
    (df.TaxiIn != "NA")
)

# Show the count of rows after filtering
print(f"Number of rows after filtering: {df.count()}")

Number of rows after filtering: 6855029


In [48]:
df.show()

+-----+---------+-------+-------------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+--------+------------+------------+--------+-------------+-----------------+
|Month|DayOfWeek|ArrTime|UniqueCarrier|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+-----+---------+-------+-------------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+--------+------------+------------+--------+-------------+-----------------+
|    1|        4|   2211|           WN|              128|           150|    116|     -14|       8|   IAD| TPA|     810|     4|      8|       0|          NA|          NA|      NA|           NA|               NA|
|    1|        4|   1002|           WN|              128|           145|    113|       2|      19|   IAD| TPA|     810|     5|     10|       0|          NA|

In [49]:
# List of columns to check
columns_to_check = ["WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay", "CarrierDelay"]

# Loop through columns and replace "NA" with 0
for column in columns_to_check:
    df = df.withColumn(
        column,
        when(col(column) == "NA", 0).otherwise(col(column))
    )

# Show the updated DataFrame
df.show()

+-----+---------+-------+-------------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+--------+------------+------------+--------+-------------+-----------------+
|Month|DayOfWeek|ArrTime|UniqueCarrier|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+-----+---------+-------+-------------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+--------+------------+------------+--------+-------------+-----------------+
|    1|        4|   2211|           WN|              128|           150|    116|     -14|       8|   IAD| TPA|     810|     4|      8|       0|           0|           0|       0|            0|                0|
|    1|        4|   1002|           WN|              128|           145|    113|       2|      19|   IAD| TPA|     810|     5|     10|       0|           0|

In [50]:
df.describe()

DataFrame[summary: string, Month: string, DayOfWeek: string, ArrTime: string, UniqueCarrier: string, ActualElapsedTime: string, CRSElapsedTime: string, AirTime: string, ArrDelay: string, DepDelay: string, Origin: string, Dest: string, Distance: string, TaxiIn: string, TaxiOut: string, Diverted: string, CarrierDelay: string, WeatherDelay: string, NASDelay: string, SecurityDelay: string, LateAircraftDelay: string]

In [51]:
from pyspark.sql import functions as F

# List of columns to exclude (non-numeric columns)
exclude_columns = ["UniqueCarrier", "Origin", "Dest"]

# Get the list of all columns in the DataFrame
all_columns = df.columns

# Identify columns that need to be cast to float (i.e., all columns except the ones to exclude)
columns_to_cast = [col for col in all_columns if col not in exclude_columns]

# Loop through the columns that need to be cast and apply the transformation
for column in columns_to_cast:
    df = df.withColumn(column, F.col(column).cast("float"))

# Show the schema to verify the changes
df.printSchema()


root
 |-- Month: float (nullable = true)
 |-- DayOfWeek: float (nullable = true)
 |-- ArrTime: float (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- ActualElapsedTime: float (nullable = true)
 |-- CRSElapsedTime: float (nullable = true)
 |-- AirTime: float (nullable = true)
 |-- ArrDelay: float (nullable = true)
 |-- DepDelay: float (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: float (nullable = true)
 |-- TaxiIn: float (nullable = true)
 |-- TaxiOut: float (nullable = true)
 |-- Diverted: float (nullable = true)
 |-- CarrierDelay: float (nullable = true)
 |-- WeatherDelay: float (nullable = true)
 |-- NASDelay: float (nullable = true)
 |-- SecurityDelay: float (nullable = true)
 |-- LateAircraftDelay: float (nullable = true)


In [52]:
# Create a new column for TotalDelay by summing delay-related columns
df = df.withColumn(
    "TotalDelay",
    F.coalesce(F.col("ArrDelay"), F.lit(0)) +
    F.coalesce(F.col("DepDelay"), F.lit(0)) +
    F.coalesce(F.col("CarrierDelay"), F.lit(0)) +
    F.coalesce(F.col("WeatherDelay"), F.lit(0)) +
    F.coalesce(F.col("NASDelay"), F.lit(0)) +
    F.coalesce(F.col("SecurityDelay"), F.lit(0)) +
    F.coalesce(F.col("LateAircraftDelay"), F.lit(0))
)

# Show the DataFrame with the new TotalDelay column (optional)
df.show(5)

+-----+---------+-------+-------------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+--------+------------+------------+--------+-------------+-----------------+----------+
|Month|DayOfWeek|ArrTime|UniqueCarrier|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|TotalDelay|
+-----+---------+-------+-------------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+--------+------------+------------+--------+-------------+-----------------+----------+
|  1.0|      4.0| 2211.0|           WN|            128.0|         150.0|  116.0|   -14.0|     8.0|   IAD| TPA|   810.0|   4.0|    8.0|     0.0|         0.0|         0.0|     0.0|          0.0|              0.0|      -6.0|
|  1.0|      4.0| 1002.0|           WN|            128.0|         145.0|  113.0|     2.0|    19.0|   IAD| TPA|  

In [53]:
train_df, test_df = df.randomSplit([0.8, 0.2], seed=1234)

In [54]:
from pyspark.ml.feature import VectorAssembler

# Use VectorAssembler to combine features (same as before, or adjust based on your features)
feature_columns = ["DepDelay", "Distance", "TaxiIn", "TaxiOut", "CarrierDelay", "WeatherDelay"]

# Create the VectorAssembler
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Prepare the data by transforming the train and test datasets
train_df = assembler.transform(train_df)
test_df = assembler.transform(test_df)

In [55]:
from pyspark.ml.regression import LinearRegression

# Initialize the Linear Regression model to predict TotalDelay
lr = LinearRegression(featuresCol="features", labelCol="TotalDelay")

# Train the model on the training data
lr_model = lr.fit(train_df)

# Make predictions on the test data
predictions = lr_model.transform(test_df)

# Show predictions (optional)
predictions.select("TotalDelay", "prediction").show(10)

24/12/27 00:22:29 WARN Instrumentation: [2605afec] regParam is zero, which might cause numerical instability and overfitting.
24/12/27 00:22:44 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/12/27 00:22:44 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/12/27 00:22:50 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


+----------+-------------------+
|TotalDelay|         prediction|
+----------+-------------------+
|     129.0| 108.50444516572766|
|      -1.0|-12.004645453851518|
|      -2.0| 29.707258222648562|
|     253.0| 251.04898251237773|
|      21.0|  21.99608317994369|
|       2.0| 14.864066518797607|
|     123.0| 132.15877733500258|
|     408.0|  378.1795588250817|
|      90.0|  67.07290398536061|
|      14.0| 19.865080315753367|
+----------+-------------------+


In [56]:
from pyspark.ml.evaluation import RegressionEvaluator

# Initialize RegressionEvaluator for RMSE and R2
evaluator = RegressionEvaluator(labelCol="TotalDelay", predictionCol="prediction", metricName="rmse")

# Calculate RMSE and R2
rmse = evaluator.evaluate(predictions)
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

print(f"RMSE: {rmse}")
print(f"R2: {r2}")

RMSE: 17.50634561433547
R2: 0.9736636641350807
